# AIT724 Project - pull tweets
# Melissa Cirtain

#### Deliverables:  10-20k tweets

In [28]:
import os
import requests
import json
import pandas as pd
from pandas import json_normalize
from datetime import datetime

# Instructions to run:
1. enter your twitter token value in the cell below
1. enter your topic value (what you want to search for) in the cell below
  - Pull 5000 tweets per topic, for instance 5k **"maga"** and 5k **"statehood"**.  
  - Melissa:  maga, statehood, #chrisrocklive
  - Sarik: tyre, chatgpt, russia, ukraine. 
  - (5000 each, no time limits)
1. run this noteboook until you have 5000 tweets on your topic.  You may need to take a 15-minute break between runs to not cap out.  References below explain limitations.
1. upload your data files to https://github.com/sareek/NLP_on_Social_Media_Data.  

In [58]:
my_token = "ADD YOUR TWITTER TOKEN HERE"
# my_topic = "obama"
# my_topic = "maga"
# my_topic = "statehood"
my_topic = "#chrisrocklive"



### Helper functions and setup

In [47]:
# Set the token, which you specified above
os.environ['TOKEN'] = my_token 


def auth():
    '''return the Bearer Token (which should be set in the env)'''
    return os.getenv('TOKEN')


def create_headers(bearer_token):
    '''return the authorization header with bearer token'''
    headers = {"Authorization": f"Bearer {bearer_token}"}
    return headers


def create_url(keyword, max_results=10):  
    '''Generate the URL with query baked in and return search results
    There is an API limit of 200(?) tweets, and time limits as well. 
    20 requests per 15 min.
    '''
    search_url = 'https://api.twitter.com/2/tweets/search/recent' # use search recent not search all (no access)
    
    # define query parameters:
    query_params = {
        'query': keyword,
        'max_results': max_results,
        #'start_time': "2023-03-06T00:00:00Z",  # lose geo and other fields when we limit
        #'end_time': "2023-03-06T11:59:59Z",
        'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
        'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,' \
                        'created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
        'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
        'place.fields':'full_name,id,country,country_code,geo,name,place_type',
        'next_token': {}
    }
    
    return (search_url, query_params)


def connect_to_endpoint(url, headers, params, next_token=None):
    '''given search arguments, connect to API and return response'''
    params['next_token'] = next_token  # params obbject received fromo create_url()
    response = requests.request('GET', url, headers=headers, params=params)
    print(f'Endpiont Response Code: {response.status_code}')
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
        
    return response.json()  # return response as JSON


# Revise connect_to_endpoint so I can iterate over pages
def connect_to_endpoint_2(url, headers, params):
    '''given search arguments, connect to API and return response'''
    #params['next_token'] = next_token  # params object received fromo create_url()
    response = requests.request('GET', url, headers=headers, params=params)
    print(f'Endpiont Response Code: {response.status_code}')
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
        
    return response.json()  # return response as JSON



def collect_tweets_to_df(keyword:str, tweets_count=5000):  # TODO: max_results here should be variable, but call to create_url() maxes at 100.
    '''Return a pandas df containing results of a query based on
    provided keyword and a parameters list.  
    '''
    max_results_per_free_request = 100
    headers = create_headers(auth())
    search_url, query_params = create_url(keyword, max_results_per_free_request)
    my_data = []
    next_token = None  # to keep getting more tweets if available
    
    while len(my_data) < tweets_count:
        # keep getting responses until we have needed number of tweets
        my_response = connect_to_endpoint_2(search_url, headers, query_params)
        
        # collect each set of responses until we have them all or meet our requirement
        my_data.extend(my_response['data'])  
        
        # get next page of tweets, if available
        if 'next_token' in my_response['meta'].keys():
            print(f'next_token found: {my_response["meta"]["next_token"]}')
            #return None, my_response
            
            # get next page of tweets
            query_params['next_token'] = my_response['meta']['next_token']

        
    # convert to dataframe keeping the interesting fields
    fields_to_keep = ['text','created_at', 'id', 'conversation_id', 'referenced_tweets','lang',
                      'author_id','public_metrics.retweet_count','public_metrics.reply_count',
                      'public_metrics.like_count','public_metrics.quote_count',
                      'public_metrics.impression_count'
                     ]
    
    #df = json_normalize(my_response['data'])
    df = json_normalize(my_data)
    
    # Keeping all the data for now.  We can reduce later if needed.
    #df = df[fields_to_keep]  # warning - will break if twitter changes their API/fieldnames
    return df, my_response



# Pull the tweets

Just run the following cell and you'll pull some tweets.  You should take a break between runs.  I will add timestamps to the saved files so you don't accidentally overwrite your output.  Run until you have 5000 tweets for both topics, then upload the files to github.

In [59]:
# try it out
#keyword = "memphis lang:en -is:retweet"
#my_df, my_response = collect_tweets_to_df(keyword)

my_keyword = f'{my_topic} lang:en'  # by default keyword search is case-insensitive; retweets appear truncated

my_df, my_response = collect_tweets_to_df(my_keyword)
print(f'\n\n\n***** PULLED {my_df.shape[0]} tweets *****\n\n')
display(my_df.head())

# write df to tsv
timestamp = str(datetime.now()).replace(' ', '_').replace(':', '.')
file_topic = my_topic.replace(' ', '_')
output_path = f'{file_topic}_project_tweets.{timestamp}.tsv'

my_df.to_csv(output_path, sep='\t')
print(f'Wrote data to {output_path}')

# write also to json (just in case)
output_path = f'{file_topic}_project_tweets.{timestamp}.json'

my_df.to_json(output_path)
print(f'Wrote json data to {output_path}')


Endpiont Response Code: 200
next_token found: b26v89c19zqg8o3fqka060r86c29c3pazq61axbr4oj99
Endpiont Response Code: 200
next_token found: b26v89c19zqg8o3fqka05yllm6ptdxukpjld740jzqqnx
Endpiont Response Code: 200
next_token found: b26v89c19zqg8o3fqka05wh6cb5y7vgp483su2u5wyfst
Endpiont Response Code: 200
next_token found: b26v89c19zqg8o3fqka05udnfrnlxjb0dgo17p8hiblod
Endpiont Response Code: 200
next_token found: b26v89c19zqg8o3fqka05uc5d2p1et0k6dfeu3vj25r3x
Endpiont Response Code: 200
next_token found: b26v89c19zqg8o3fqka05s8mdeyd5gip8qvi63eirnybh
Endpiont Response Code: 200
next_token found: b26v89c19zqg8o3fqka05s74mvtzxbzeqhhs5z8oi4ku5
Endpiont Response Code: 200
next_token found: b26v89c19zqg8o3fqka05q47520rf0v0cahodhurz2hz1
Endpiont Response Code: 200
next_token found: b26v89c19zqg8o3fqka05q3lh7ixdx1izqrnjzp097bst
Endpiont Response Code: 200
next_token found: b26v89c19zqg8o3fqka05q2pei9j1ti2diem1914532pp
Endpiont Response Code: 200
next_token found: b26v89c19zqg8o3fqka05o0dhlc06u0agp

,conversation_id,lang,reply_settings,created_at,text,author_id,referenced_tweets,id,edit_history_tweet_ids,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,in_reply_to_user_id,geo.place_id,withheld.copyright,withheld.country_codes
0,1633954063934009354,en,everyone,2023-03-09T22:13:00.000Z,RT @Infantry_bucky: He’s lucky a #ChrisRockLiv...,1519164980582653953,"[{'type': 'retweeted', 'id': '1633938373529292...",1633954063934009354,[1633954063934009354],8,0,0,0,0,NaN,NaN,NaN,NaN
1,1633954058212876289,en,everyone,2023-03-09T22:12:59.000Z,RT @1_ofakindnocap: Chris Rock: “we all been c...,21575184,"[{'type': 'retweeted', 'id': '1632283297588948...",1633954058212876289,[1633954058212876289],616,0,0,0,0,NaN,NaN,NaN,NaN
2,1633951267423768578,en,everyone,2023-03-09T22:01:54.000Z,RT @1_ofakindnocap: Chris Rock: “we all been c...,360633018,"[{'type': 'retweeted', 'id': '1632283297588948...",1633951267423768578,[1633951267423768578],616,0,0,0,0,NaN,NaN,NaN,NaN
3,1633950853626318850,en,everyone,2023-03-09T22:00:15.000Z,RT @rolandsmartin: Working out and watching th...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632825595473149...",1633950853626318850,[1633950853626318850],14,0,0,0,0,NaN,NaN,NaN,NaN
4,1633950824664645634,en,everyone,2023-03-09T22:00:08.000Z,RT @rolandsmartin: Workout done. I’ll have a f...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632833853021728...",1633950824664645634,[1633950824664645634],8,0,0,0,0,NaN,NaN,NaN,NaN


Wrote data to #chrisrocklive_project_tweets.2023-03-09_17.16.18.676751.tsv
Wrote json data to #chrisrocklive_project_tweets.2023-03-09_17.16.18.676751.json


In [55]:
my_df['geo.place_id'].unique()
my_df['geo.place_id'].value_counts()
my_df.loc[0, 'text']

'RT @Wasred4skins: Can anyone name a MAGA Republican that wants to defund the police?'

In [42]:
# Example opening datasets:
new_df = pd.read_json('maga_project_tweets.2023-03-08_21.15.55.808461.json')
display(new_df.head())

other_df = pd.read_csv('maga_project_tweets.2023-03-08_20.58.51.129896.tsv', delimiter='\t')
other_df.head()

,reply_settings,id,lang,text,conversation_id,author_id,referenced_tweets,edit_history_tweet_ids,created_at,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,in_reply_to_user_id,geo.place_id
0,everyone,1633648438993883136,en,RT @GabiNga1: @keith0sta @827js @1mir_r @45tf5...,1633648438993883136,1298715879673274368,"[{'type': 'retweeted', 'id': '1633505762881282...",[1633648438993883136],2023-03-09 01:58:34+00:00,106,0,0,0,0,NaN,None
1,everyone,1633648437798522880,en,RT @OccupyDemocrats: BREAKING: MAGA Gov. Sarah...,1633648437798522880,615777223,"[{'type': 'retweeted', 'id': '1633546143337680...",[1633648437798522881],2023-03-09 01:58:33+00:00,2300,0,0,0,0,NaN,None
2,everyone,1633648433977335808,en,RT @OccupyDemocrats: BREAKING: MAGA Gov. Sarah...,1633648433977335808,1393026118303854592,"[{'type': 'retweeted', 'id': '1633546143337680...",[1633648433977335808],2023-03-09 01:58:33+00:00,2300,0,0,0,0,NaN,None
3,everyone,1633648433507573760,en,@ybarrap We demand justice when cops shooting ...,1633564967164100608,739248656,"[{'type': 'replied_to', 'id': '163356496716410...",[1633648433507573763],2023-03-09 01:58:32+00:00,0,0,0,0,0,17180761.0,None
4,everyone,1633648424569679872,en,RT @TheRickWilson: They're captives of in the ...,1633648424569679872,899686918719381504,"[{'type': 'retweeted', 'id': '1633530595572842...",[1633648424569679872],2023-03-09 01:58:30+00:00,151,0,0,0,0,NaN,None


,Unnamed: 0,reply_settings,id,lang,text,conversation_id,author_id,referenced_tweets,edit_history_tweet_ids,created_at,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,in_reply_to_user_id,geo.place_id
0,0,everyone,1633648438993883136,en,RT @GabiNga1: @keith0sta @827js @1mir_r @45tf5...,1633648438993883136,1298715879673274370,"[{'type': 'retweeted', 'id': '1633505762881282...",['1633648438993883136'],2023-03-09T01:58:34.000Z,106,0,0,0,0,NaN,NaN
1,1,everyone,1633648437798522881,en,RT @OccupyDemocrats: BREAKING: MAGA Gov. Sarah...,1633648437798522881,615777223,"[{'type': 'retweeted', 'id': '1633546143337680...",['1633648437798522881'],2023-03-09T01:58:33.000Z,2300,0,0,0,0,NaN,NaN
2,2,everyone,1633648433977335808,en,RT @OccupyDemocrats: BREAKING: MAGA Gov. Sarah...,1633648433977335808,1393026118303854594,"[{'type': 'retweeted', 'id': '1633546143337680...",['1633648433977335808'],2023-03-09T01:58:33.000Z,2300,0,0,0,0,NaN,NaN
3,3,everyone,1633648433507573763,en,@ybarrap We demand justice when cops shooting ...,1633564967164100615,739248656,"[{'type': 'replied_to', 'id': '163356496716410...",['1633648433507573763'],2023-03-09T01:58:32.000Z,0,0,0,0,0,17180761.0,NaN
4,4,everyone,1633648424569679872,en,RT @TheRickWilson: They're captives of in the ...,1633648424569679872,899686918719381506,"[{'type': 'retweeted', 'id': '1633530595572842...",['1633648424569679872'],2023-03-09T01:58:30.000Z,151,0,0,0,0,NaN,NaN


# References and Notes

#### References Used
- [Towards Data Science Twitter API v2 Post](https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a)
- [Twitter developer Recent API docs](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent#tab1)
- [Building Queries API 2 docs](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query)
- [Twitter Dev API - enriched tweet objects](https://developer.twitter.com/en/docs/twitter-api/enterprise/data-dictionary/native-enriched-objects/tweet)
- [Recent search params](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent)
- [Stack Overflow, avoiding truncation](https://stackoverflow.com/questions/38717816/twitter-api-text-field-value-is-truncated)

#### URL/Query builder

- [Building Queries API 2 docs](https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query)
- [Twitter Dev API](https://developer.twitter.com/en/docs/twitter-api/enterprise/data-dictionary/native-enriched-objects/tweet)
- [Recent search params](https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent)
- avoid truncation: https://stackoverflow.com/questions/38717816/twitter-api-text-field-value-is-truncated
- `(400, '{"errors":[{"parameters":{"tweet.truncated":["false"]},"message":"The query parameter [tweet.truncated] is not one of [query,start_time,end_time,since_id,until_id,max_results,next_token,pagination_token,sort_order,expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields]"}],"title":"Invalid Request","detail":"One or more parameters to your request was invalid.","type":"https://api.twitter.com/2/problems/invalid-request"}')`

#### Overcoming API Limitation

- [Official twitter limits documentation](https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/faq#:~:text=900%20requests%2F15%2Dmin%20window,%2Dhour%20window%20(application%20level))

I have found two ways to get more than the 100 tweets allowed in a single request.  

1. `my_response['meta']['next_token']` - Control for pagination, when more than a single page of tweets are available within a response.  This is passed in with the request as one of the params [gist](https://gist.githubusercontent.com/AndrewEdward37/8309d5701af8b5414498f60aafd68a5d/raw/d7fbf33a61d58943348deb3a20d94284c4deffc4/connect.py): 
```
params['next_token'] = next_token
responose = requests.request("GET", url, headers=headers, params=parms)
```

1. Windowing over time ranges using `start_time` and `end_time` in query parameters.  Type should be ISO 8601 date, i.e. `YYYY-MM-DDTHH:mm:ssZ`.  This sets the newest, most recent UTC timestap to which the Tweets will be provided. 